# **Tweeter Sentiment Analysing Model using BERT**

In [3]:
!pip3 install tensorflow
!pip3 install transformers
!pip3 install scikit-learn

In [4]:
import numpy as np
import pandas as pd
import sklearn

import tensorflow as tf
import transformers
#tqdm is a progress bar
from tqdm import tqdm

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/twitter_final_Dataset.csv", encoding='latin-1')
df.head()

,Unnamed: 0,tweet,sentiment
0,0,@Microsoft Why do I pay for WORD when it funct...,0.0
1,1,"CSGO matchmaking is so full of closet hacking,...",0.0
2,2,Hi @EAHelp Iâve had Madeleine McCann in my c...,0.0
3,3,Thank you @EAMaddenNFL!! \n\nNew TE Austin Hoo...,1.0
4,4,"Rocket League, Sea of Thieves or Rainbow Six: ...",1.0


In [7]:
df['sentiment_new'] = df['sentiment'].replace([0.0], 'Negative')
df['sentiment_new'] = df['sentiment_new'].replace([1.0], 'Poistive')
df.head

<bound method NDFrame.head of          Unnamed: 0                                              tweet  \
0                 0  @Microsoft Why do I pay for WORD when it funct...   
1                 1  CSGO matchmaking is so full of closet hacking,...   
2                 2  Hi @EAHelp Iâve had Madeleine McCann in my c...   
3                 3  Thank you @EAMaddenNFL!! \n\nNew TE Austin Hoo...   
4                 4  Rocket League, Sea of Thieves or Rainbow Six: ...   
...             ...                                                ...   
1725686     1725686  modi promised 2014 lok sabha elections that be...   
1725687     1725687  save your agenda peddling â had with terror ...   
1725688     1725688  did you cover her interaction forum where she ...   
1725689     1725689  there big project came into india modi dream p...   
1725690     1725690  have you ever listen about like gurukul where ...   

         sentiment sentiment_new  
0              0.0      Negative  
1          

In [8]:
df1 = df.dropna()
df1.head

<bound method NDFrame.head of          Unnamed: 0                                              tweet  \
0                 0  @Microsoft Why do I pay for WORD when it funct...   
1                 1  CSGO matchmaking is so full of closet hacking,...   
2                 2  Hi @EAHelp Iâve had Madeleine McCann in my c...   
3                 3  Thank you @EAMaddenNFL!! \n\nNew TE Austin Hoo...   
4                 4  Rocket League, Sea of Thieves or Rainbow Six: ...   
...             ...                                                ...   
1725686     1725686  modi promised 2014 lok sabha elections that be...   
1725687     1725687  save your agenda peddling â had with terror ...   
1725688     1725688  did you cover her interaction forum where she ...   
1725689     1725689  there big project came into india modi dream p...   
1725690     1725690  have you ever listen about like gurukul where ...   

         sentiment sentiment_new  
0              0.0      Negative  
1          

In [9]:
# Loading the BERT Classifier and Tokenizer along with Input module
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
df1.describe()

,Unnamed: 0,sentiment
count,1.725330e+06,1.725330e+06
mean,8.630206e+05,5.052402e-01
std,4.980684e+05,4.999727e-01
min,0.000000e+00,0.000000e+00
25%,4.316932e+05,0.000000e+00
50%,8.630255e+05,1.000000e+00
75%,1.294358e+06,1.000000e+00
max,1.725690e+06,1.000000e+00


In [12]:
# changing positive and negative into numeric values

def cat2num(value):
    if value=='positive':
        return 1
    else:
        return 0

df1['sentiment_new']  =  df1['sentiment_new'].apply(cat2num)
train = df1[:10000]
test = df1[11000:12000]

<ipython-input-12-83b1e01b8310>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['sentiment_new']  =  df1['sentiment_new'].apply(cat2num)


In [13]:
def convert_data_to_examples(train, test, text, sentiment):
    train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[text],
                                                          label = x[sentiment]), axis = 1)

    validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[text],
                                                          label = x[sentiment]), axis = 1,)

    return train_InputExamples, validation_InputExamples

train_InputExamples, validation_InputExamples = convert_data_to_examples(train,  test, 'tweet',  'sentiment')

In [14]:
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in tqdm(examples):
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,    # Add 'CLS' and 'SEP'
            max_length=max_length,    # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],input_dict["token_type_ids"], input_dict['attention_mask'])
        features.append(InputFeatures( input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label) )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'tweet'
LABEL_COLUMN = 'sentiment'

In [15]:
train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

  0%|          | 0/10000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 10000/10000 [00:05<00:00, 1752.66it/s]


In [16]:
validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

100%|██████████| 1000/1000 [00:01<00:00, 901.19it/s]


In [20]:
#If you are on a new Mac, change 'tf.keras.optimizers.Adam' to 'tf.keras.optimizers.legacy.Adam' it'll likely be faster
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=3e-8),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=50, validation_data=validation_data)

Epoch 1/50
626/626 [==============================] - 546s 827ms/step - loss: 0.6976 - accuracy: 0.5004 - val_loss: 0.6719 - val_accuracy: 0.6130
Epoch 2/50
626/626 [==============================] - 519s 829ms/step - loss: 0.6929 - accuracy: 0.5176 - val_loss: 0.6726 - val_accuracy: 0.6210
Epoch 3/50
626/626 [==============================] - 518s 827ms/step - loss: 0.6892 - accuracy: 0.5336 - val_loss: 0.6723 - val_accuracy: 0.6360
Epoch 4/50
626/626 [==============================] - 518s 827ms/step - loss: 0.6875 - accuracy: 0.5436 - val_loss: 0.6710 - val_accuracy: 0.6500
Epoch 5/50
626/626 [==============================] - 518s 828ms/step - loss: 0.6844 - accuracy: 0.5604 - val_loss: 0.6684 - val_accuracy: 0.6750
Epoch 6/50
626/626 [==============================] - 518s 828ms/step - loss: 0.6804 - accuracy: 0.5832 - val_loss: 0.6651 - val_accuracy: 0.6990
Epoch 7/50
626/626 [==============================] - 519s 829ms/step - loss: 0.6777 - accuracy: 0.5961 - val_loss: 0.6602 -

In [23]:
pred_sentences = ['Finally, the semester is about to get over', 'Happy' ,'Why does it always rain on weekends','Tiger 3 is the worst movie I have seen','Good time to eat some Tacos']

In [28]:
 # we are tokenizing before sending into our trained model
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
#axis=-1, this means that the index that will be returned by argmax will be taken from the *last* axis.
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
    print(pred_sentences[i], ": ", labels[label[i]])

Finally, the semester is about to get over :  Positive
Happy :  Positive
Why does it always rain on weekends :  Negative
Tiger 3 is the worst movie I have seen :  Negative
Good time to eat some Tacos :  Positive


The model is at roughly 57% accuracy and that can be said by watching the results above.